In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pwd

In [ ]:
!ls -al ../input

In [ ]:
trn_full = pd.read_csv('../input/santander-product-recommendation/train_ver2.csv.zip')

In [ ]:
trn = trn_full.sample(n=10000)

In [ ]:
# 데이터 값 실수. 소수점 자리 설정
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
trn.shape

In [ ]:
trn.head()

In [ ]:
# 모든 변수의 미리보기
for col in trn.columns:
    print('{}\n'.format(trn[col].head()))

In [ ]:
trn.info()

In [ ]:
trn.ind_nomina_ult1.value_counts()

In [ ]:
trn['ind_recibo_ult1'].value_counts()

In [ ]:
trn.isnull().sum()

In [ ]:
# 고객 관련 변수 중 수치형 변수 살펴보기
num_cols = [col for col in trn.columns[:24] if trn[col].dtype in ['int64','float64']]
trn[num_cols].describe()

### dtype 'object' 인데 왜 trn[col].dtype in ['o'] 소문자 o를 쓰면 오류 발생할까? 대문자 'O' 정상처리

In [ ]:
# 고객 관련 변수 중 범주형 변수 살펴보기
cat_cols = [col for col in trn.columns[:24] if trn[col].dtype in ['O']]
trn[cat_cols].describe()

In [ ]:
# 범주형 변수의 고유값 직접 출력해보기
for col in cat_cols:
    uniq = np.unique(trn[col].astype(str))
    print('-' * 50)
    print('# col {}, n_uniq {}, uniq{}'.format(col,len(uniq), uniq))

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
skip_cols = ['ncodpers', 'renta']
for col in trn.columns:
    # 출력이 너무 시간이 많이 걸리는 두 변수는 skip한다.
    if col in skip_cols:
        continue
        
    # 보기 편하게 영역 구분과 변수명 출력한다.
    print('-' * 50)
    print('col :', col)
    
    # 그래프 크기(figsize) 설정
    f, ax = plt.subplots(figsize=(20, 15))
    # seaborn을 사용한 막대 그래프를 생성한다. alpha는 투명도, countplot은 histogram
    sns.countplot(x=col, data=trn, alpha=0.5)
    # show() 함수를 통해 시각화한다.
    plt.show()

In [ ]:
# 월별 금융 제품 보유 데이터를 누적 막대 그래프로 시각화하기
months = trn['fecha_dato'].unique().tolist()
# 제품 변수 24개를 추출한다.
label_cols = trn.columns[24:].tolist()

label_over_time = []
for i in range(len(label_cols)):
    # 매월 각 제품의 총합을 groupby(..).agg('sum')으로 계산하여, label_sum에 저장한다.
    label_sum = trn.groupby(['fecha_dato'])[label_cols[i]].agg('sum')
    label_over_time.append(label_sum.tolist())

label_sum_over_time = []
for i in range(len(label_cols)):
    # 누적 막대 그래프를 시각화하기 위하여, n번째 제품의 총합을 1~n 번째 제품의 총합으로 만든다.
    label_sum_over_time.append(np.asarray(label_over_time[i:]).sum(axis=0))
    
# 시각화를 위하여 색깔을 정의한다.
color_list = ['#F5B7B1','#D2B4DE','#AED6F1','#A2D9CE','#ABEBC6','#F9E79F','#F5CBA7','#CCD1D1']
    
# 그림 크기를 사전에 정의한다.
f, ax = plt.subplots(figsize=(30,15))
for i in range(len(label_cols)):
# 24개 제품에 대하여 Histogram을 그린다.
# x축에는 월 데이터, y축에는 누적 총합, 색깔은 8개를 번갈아 가며 사용하며, 그림의 alpha값은 0.7로 지정한다.
   sns.barplot(x=months, y=label_sum_over_time[i], color=color_list[i%8], alpha=0.7)
    
# 우측 상단에 Legend를 추가한다.
   plt.legend([plt.Rectangle((0,0),1,1, fc=color_list[i%8], edgecolor='none') for i in range(len(label_cols))],\
               label_cols, loc=1, ncol=2,prop={'size':16})   

In [ ]:
# label_sum_over_time의 값을 퍼센트 단위로 변환한다. 월마다 최대값으로 나누고 100을 곱해준다.
label_sum_percent = (label_sum_over_time / (1.*np.asarray(label_sum_over_time).max(axis=0))) * 100

# 앞선 코드와 동일한 시각화 실행 코드이다.
f, ax = plt.subplots(figsize=(30, 15))
for i in range(len(label_cols)):
    sns.barplot(x=months, y=label_sum_percent[i], color=color_list[i%8], alpha=0.7)
    
plt.legend([plt.Rectangle((0,0),1,1,fc=color_list[i%8], edgecolor='none') for i in range(len(label_cols))], label_cols, loc=1,
          ncol=2, prop={'size':16})

In [ ]:
# 24개 금융제품에 대한 '신규 구매' 데이터 생성하기
#import pandas as pd
#import numpy as np

# 훈련 데이터를 읽어온다
trn = pd.read_csv('/kaggle/input/santander-product-recommendation/train_ver2.csv.zip')

# 제품 변수를 prods에 list형태로 저장한다.
prods = trn.columns[24:].tolist()

# 날짜를 숫자로 변환하는 함수이다. 2015-01-28은 1, 2016-06-28은 18로 변환된다.
def date_to_int():
    Y, M, D = [int(a) for a in str_date.strip().split("_")]
    int_date = (int(Y) - 2015) * 12 + int(M)
    return int_date

# 날짜를 숫자로 변환하여 int_date에 저장한다.
trn['int_date'] = trn['fecha_dato'].map(date_to_int).astype(np.int8)

# 데이터를 복사하고, int_date 날짜에 1을 더하여 lag를 생성한다. 변수명에 _prev를 추가한다.
trn_lag = trn.copy()
trn_lag['int_date'] += 1
trn_lag.columns = [col + '_prev' if col not in ['ncodpers', 'int_date']
                  else col for col in trn.columns]

# 원본 데이터와 lag 데이터를 ncodper와 int_date 기준으로 합친다. lag 데이터의 int_date는 1 밀려 있기 때문에, 저번 달의 제품 정보가 삽입된다.
df_trn = trn.merge(trn_lag, on=['ncodpers','int_date'], how='left')

# 메모리 효율을 위해 불필요한 변수를 메모리에서 제거한다.
del trn, trn_lag

# 저번 달의 제품 정보가 존재하지 않을 경우를 대비하여 0으로 대체한다.
for prod in prods:
    prev = prod + '_prev'
    df_trn[prev].fillna(0, inplace=True)
    
# 원본 데이터에서의 제품 보유 여부 - lag데이터에서의 제품 보유 여부를 비교하여 신규 구매 변수 padd를 구한다.
for prod in prods:
    padd = prod + '_add'
    prev = prod + '_prev'
    df_trn[padd] = ((df_trn[prod] == 1) & (df_trn[prev] == 0)).astype(np.int8)
    
# 신규 구매 변수만을 추출하여 labels에 저장한다.
add_cols = [prod + '_add' for prod in prods]
labels = df_trn[add_cols].copy()
labels.columns = prods
labels.to_csv('../input/labels.csv', index=False)

In [ ]:
labels = pd.read_csv('../input/labels.csv').astype(int)
fecha_dato = pd.read_csv('/kaggle/input/santander-product-recommendation/train_ver2.csv.zip', usecols=['fecha_dato'])

labels['date'] = fecha_dato.fecha_dato
months = np.unique(fecha_dato.fecha_dato).tolist()
label_cols = labels.columns.tolist()[:24]

label_over_time = []
for i in range(len(label_cols)):
    label_over_time.append(labels.groupby(['data'])[label_cols[i]].agg('sum').tolist())

label_sum_over_time = []
for i in range(len(label_cols)):
    label_sum_over_time.append(np.asarray(label_over_time[i:]).sum(axis=0))

color_list = ['#F5B7B1','#D2B4DE','#AED6F1','#A2D9CE','#ABEBC6','#F9E79F','#F5CBA7','#CCD1D1']

f, ax = plt.subplots(figsize=(30, 15))
for i in range(len(label_cols)):
    sns.subplot(x=months, y=label_sum_over_time[i], color=color_list[i%8], alpha=0.7)
    
plt.legend([plt.Rectangle((0,0),1,1,fc=color_list[i%8], edgecolor='none') for i in range(len(label_cols))], 
           label_cols, loc=1, ncol=2, prop=('size':16))

In [ ]:
# label_sum_over_time의 값을 퍼센트 단위로 변환한다. 월마다 최대값으로 나누고 100을 곱해준다.
label_sum_percent = (label_sum_over_time / (1.*np.asarray(label_sum_over_time).max(axis=0))) * 100

# 앞선 코드와 동일한 시각화 실행 코드이다.
f, ax = plt.subplots(figsize=(30, 15))
for i in range(len(label_cols)):
    sns.barplot(x=months, y=label_sum_percent[i], color=color_list[i%8], alpha=0.7)
    
plt.legend([plt.Rectangle((0,0),1,1,fc=color_list[i%8], edgecolor='none') for i in range(len(label_cols))], label_cols, loc=1,
          ncol=2, prop={'size':16})